In [1]:
# default_exp utils.visualization

In [2]:
# export

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cmx
import matplotlib.patches as patches


import plotly.io as plt_io
import plotly.graph_objects as go
import plotly.offline as pyo

pyo.init_notebook_mode()

import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LogNorm

from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture

from datetime import datetime
from pathlib import Path
from typing import Optional, Dict, Tuple, List
from collections.abc import Iterable

# Configs

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [1]:
import plotly.io as plt_io
import plotly.graph_objects as go

## visualization

> Methods for visualization purposes, specially for clustering-related results.

## Utils

In [49]:
# export

# Util. method for saving pyplot plots and validate output directory

def _save_plot(out_path: str,  file_name: str, file_format: Optional[str]='png'):
    """
    Store pyplot figures veryfing containing directories
    """
    path = Path(out_path)
    
    if not path.exists():
        path.mkdir(exist_ok=True)
        
    now = datetime.now()
    ts = datetime.timestamp(now)
    
    plt.savefig(f"{str(path/file_name)}-{ts}.{file_format}", dpi=100, format=file_format)

In [ ]:
# export

def save_plotly_fig(fig: go.Figure, out_path: str, fig_name: str, img_fmt: Optional[str]='png',
                    export_additional_svg: Optional[bool]=True):
    """
    Export the given plotly Figure using the specified format
    
    :param fig: Plotly Figure to be exported
    :param out_path: str specifying the output directory
    :param fig_name: str specifying the name for the img file
    :param img_fmt:  Optional str specifying the format (extension) for the generated file
                     default: png
    
    """
    
    path = Path(out_path)    
    file_name = f"{fig_name}.{img_fmt}"

    if not path.exists():
        logging.error(f"Figure {file_name} could not be exported,\n{out_path} does not exist.")
        return
    
    full_path = path / file_name
    fig.write_image(str(full_path),  engine="kaleido")
    
    logging.info(f"Exported figure at: {str(full_path)}")
    
    if export_additional_svg:
        file_name = f"{fig_name}.svg"
        full_path = path / file_name
        fig.write_image(str(full_path), engine='kaleido')
        
        logging.info(f"Exported svg figure at: {str(full_path)}")

In [ ]:
# export

def _get_plotly_colors():
    """
    Gets a list containing multiple discrete plotly colors as a list
    """
    colors = px.colors.qualitative.Plotly
    colors = px.colors.qualitative.D3
    colors = px.colors.qualitative.Set2
    colors += px.colors.qualitative.Set3
    colors += px.colors.qualitative.Safe
    colors += px.colors.qualitative.Light24
    colors += px.colors.qualitative.Vivid
    colors += px.colors.qualitative.Pastel
    colors += px.colors.qualitative.Pastel1
    colors += px.colors.qualitative.Antique
    colors += px.colors.qualitative.Prism
    
    return colors   

In [22]:
# export

# Generate Mixed Gaussian Models with k components

def generate_distributions(feature_vectors, k):
    """
    Generate Mixed Gaussian mixture model with k components
    :param feature_vectors: Data
    :param k: Number of components to get GMM
    :return: sklearn GMM
    """
    gmm = GaussianMixture(k, covariance_type="diag")
    gmm.fit(feature_vectors)
    
    return gmm

In [ ]:
# export

def setup_data(cluster):
    """
    Get decision boundaries by means of np.meshgrid
    
    :return: Tuple (vectors, centroids, X component of mesghgrid, Y component of meshgrid, )
    """
    feature_vectors, _, centroids, _, kmeans = cluster
    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .2     # point in the mesh [x_min, x_max]x[y_min, y_max].

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = feature_vectors[:, 0].min() - 1, feature_vectors[:, 0].max() + 1
    y_min, y_max = feature_vectors[:, 1].min() - 1, feature_vectors[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    return feature_vectors, centroids, xx, yy, Z

In [39]:
# export

def setup_meshgrid(cluster, h=0.05):
    vectors, _, medoid_ids, kmedoids = cluster
    medoids = np.array([vectors[i] for i in medoid_ids])
    # h = .8     # point in the mesh [x_min, x_max]x[y_min, y_max].

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = vectors[:, 0].min() - 1, vectors[:, 0].max() + 1
    y_min, y_max = vectors[:, 1].min() - 1, vectors[:, 1].max() + 1
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h)) # np.meshgrid(np.arange(-10, 10, h), np.arange(-10, 10, h))
    
    Z = kmedoids.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    y_ = np.arange(y_min, y_max, h)
    
    return vectors, medoids, xx, yy, Z, y_

In [37]:
# export

# Change to providing it the actual samples from the distribution instead of sampling from it

def gmm_kl(gmm_p, gmm_q, n_samples=10**5):
    """
    Compute the KL-divergence value for 2 given
    Gaussian Mixture models
    
    :param gmm_p: 1st GMM model
    :param gmm_q: 2nd GMM model
    
    :return: Float indicating KL-divergence value
    """
    # From https://stackoverflow.com/a/26079963/5768407
    X = gmm_p.sample(n_samples)[0]
    log_p_X = gmm_p.score_samples(X)
    log_q_X = gmm_q.score_samples(X)
    return log_p_X.mean() - log_q_X.mean()

## Configuration

In [19]:
params = {
    "result_plots_path": "/tf/main/dvc-ds4se/results/images"
}

## Plotting methods

Plotting data distribution

In [ ]:
# export

def plot_data_distribution_2d(feat_vectors: np.ndarray,
                               dist_name: str,
                               out_path: str,
                               color: Optional[str]="lightskyblue",
                               img_fmt: Optional[str]="png",
                               show: Optional[bool]=True,
                               export: Optional[bool]=False):
    """
    Plot the data distribution reduced using UMAP (2 dimensions)
    
    :param feat_vectors: Vectors containing data to be plotted
    :param dist_name: String identifying the data distribution
    :param out_path: String specifying the location to store the generated figures
    :param color: String indicating the color of markers (according to plotly), default=lightskyblue
    :param img_fmt: Format of the exported img file
    :param show: (Optional) Bool indicating whether or not show the plot, default=True
    :param export: (Optional) Bool indicating whether or not export the figure, default=False
    """
    
    
    x1 = feat_vectors[:, 0]
    x2 = feat_vectors[:, 1]
    
    layout = go.Layout(
        title=f"Data distribution for {dist_name}",
        xaxis=dict(
            title="UMAP Dim.1"
        ),
        yaxis=dict(
            title="UMAP Dim.2"
        )
    )
    fig = go.Figure(data=go.Scatter(
        x = x1,
        y = x2,
        mode='markers',
        marker=dict(
            size=8,
            line_width=1,
            color=color
            
        )
    ), layout=layout)
    
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=1000,height=600)
    fig.layout.template = 'plotly_white'
    
    if show:
        fig.show()
    
    if export:
        save_plotly_fig(fig, out_path, fig_name=f'DataDistribution{dist_name}', img_fmt=img_fmt)

In [ ]:
# export

def plot_overlapping_distributions_2d(clusters,
                                       out_path: str,
                                       colors: Optional[List]=['khaki', 'lightcyan'],
                                       img_fmt: Optional[str]='png',
                                       export: Optional[bool]=False,
                                       show: Optional[bool]=True,
                                      ):
    """
    Plot the distributions for 2 given distributions.
    The distributions are asumed to be obtained via UMAP reduction (2 dimensions)
    
    :param clusters: Dictionary containing cluster-related data for 2 distributions
    :param out_path: Location to export the image file
    :param colors: (Optional) List containing the 2 colors to represent the data distributions,
                    default: ['khaki', 'lightcyan']
    :param img_fmt: (Optional) File format for the exported file, default: png
    :param export: (Optional) Boolean indicating whether or not export file, default: False
    :param show: (Optional) Boolean indicating whether show the image
    """
    
    fig = go.Figure()
    models = list(clusters.keys())
    
    for i, model in enumerate(clusters):
        feat_vectors, _, _, _ = clusters[model]
        fig.add_trace(go.Scatter(
            x = feat_vectors[:, 0],
            y = feat_vectors[:, 1],
            mode='markers',
            marker=dict(
                size=5,
                line_width=0.8,
                color=colors[i]
            ),
            name=model
        ))
    
    
    fig.update_layout(
        legend=dict(
            yanchor="top",
            # y=0.99,
            xanchor="left",
            #x=0.01
        ),
        title=f'Overlapping distributions for {models[0]} - {models[1]}',
        xaxis=dict(
            title="UMAP Dim.1"
        ),
        yaxis=dict(
            title="UMAP Dim.2"
        )
    )
    fig.layout.template = 'plotly_white'   
    
    if show:
        fig.show()
    
    if export:
        save_plotly_fig(fig, out_path, f"Overlap-distr-{models[0]}-{models[1]}", img_fmt=img_fmt)

In [ ]:
# export

def _visualize_gmm_2d(points, w, mu, stdev, model_id,
                       fig,
                       export: Optional[bool]=False,
                       fixed_color: Optional[str]=None):
    """
    Plot the GMMs for a given model
    
    :param points: Vectors of reduced data
    :param w:
    :param: mu:
    :param stdev:
    :param model_id:
    :param export: Optional[bool] indicating whether or not export the figure
    :param fixed_color: Optional[str] indicating the color to apply to all generated GMMs
                        representations (ellipses)
    
    """
    
    colors = _get_plotly_colors()
    
    n_gaussians = mu.shape[1]
    
    if n_gaussians > len(colors):
        msg = "Cannot plot the provided ammount of GMMs"
        logging.error(msg)
        raise Exception(msg)
        
    N = int(np.round(points.shape[0]/n_gaussians))
    
    for i in range(n_gaussians):
        for j in range(4):
            center = mu[:, i]
            std1 = stdev[0, i]
            std2 = stdev[1, i]
            
            width = (j+1) * stdev[0, i]
            height = (j+1) *  stdev[1, i]
            
            # print(f"center: {center}, std1: {std1}, std2: {std2}")
            
            x0 = center[0] - (width/2)
            x1 = center[0] + (width/2)
            y0 = center[1] - (height/2)
            y1 = center[1] + (height/2)
            
            gmm_color = fixed_color if fixed_color is not None else colors[i]
            
            fig.add_shape(type="circle",
                          xref="x", yref="y",
                          x0=x0, y0=y0, x1=x1, y1=y1,
                          fillcolor=gmm_color,
                          opacity=0.4,
                          line=dict(color=gmm_color, width=12)
            )
            
    fig.update_layout(
        margin=dict(l=20, r=20, b=100),
        height=700, width=700)
        
    return fig

In [ ]:
# export

def plot_gmm_2d_single_distribution(cluster_data: Tuple, model: str,
                                     out_path: str, export: Optional[bool]=False):
    """
    Plot a single distribution of datapoints alongside with the calculated GMMs
    
    :param cluster_data: Tuple containing the data for a cluster (vectors, _, centroids, _)
    :param model: Str indicating the name of the distribution
    :param out_path: Str indicating the location for the generated figure
    :param export: Optional[bool] indicating whether or not to export the figure
    
    """
    gmms = {}
    feat_vectors, _, centroid_ids, _ = cluster_data

    
    
    
    fig = go.Figure(data=go.Scatter(
        x = feat_vectors[:, 0],
        y =  feat_vectors[:, 1],
        mode='markers',
        marker=dict(
            size=5,
            line_width=0.4,
            color='rgb(255, 255, 204)'
            )
        ),
        layout=dict(
            title=f"Gaussian Mixture Models for {model}",
            xaxis=dict(
                title="UMAP Dim.1"
            ),
            yaxis=dict(
                title="UMAP Dim.2"
            )
        )
    )
    
    gmm = generate_distributions(feat_vectors, len(centroid_ids))
    gmms[model] = gmm
    
    fig = _visualize_gmm_2d(feat_vectors, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, fig)
    fig.show()
    # Matplotlib implementation
#     plt.figure(figsize = (8, 8))
#     plt.title(f'Gaussian Mixture Models for {model}')
#     plt.plot(feat_vectors[:, 0], feat_vectors[:, 1], 'c.', markersize = 4, zorder = -1)
    
#     __visualize_2D_gmm(feat_vectors, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, 'blue')
    
#     x_min = feat_vectors[:, 0].min() - 1
#     x_max = feat_vectors[:, 0].max() + 1
        
#     y_min = feat_vectors[:, 1].min() - 1
#     y_max = feat_vectors[:, 1].max() + 1
    
#     plt.xlim(x_min, x_max)
#     plt.ylim(y_min, y_max)
    
#     plt.grid(True)

In [ ]:
# export

def plot_overlapping_gmms(clusters: Dict, out_path: str, export: Optional[bool]=True,
                           scatter_colors: Optional[List]=['palegreen', 'lightskyblue'],
                           gmm_colors: Optional[List]=['rgb(242, 183, 1)', 'rgb(231, 41, 138)']
                          ) -> Tuple[float, float]:
    """
    Plot the overlapping distributions/GMMS for a couple of sample sets
    
    :param cluster: Dictionary containing cluster data for 2 sample sets
    :param out_path: Location to store generated figures
    :param export: Optional[bool] indicating whether or not to store the figures
    :param scatter_colors: Optional[List] indicating colors for plotting distribution datapoints
    :param gmm_color: Optional[List] indicating colors for plotting GMMs data
    
    :return: Tuple(float, float) containing KL divergence computation for the givent distributions
    """
    gmms = {}
    
    models = list(clusters.keys())
    model1 = models[0]
    model2 = models[1]
    
    fig = go.Figure(layout=dict(title=f"Overlapping GMMs {model1}-{model2}"))
    
    for i, model in enumerate(clusters):
        feat_vectors, _, centroid_ids, _ = clusters[model]
        fig.add_trace(go.Scatter(
            name=model,
            x = feat_vectors[:, 0],
            y =  feat_vectors[:, 1],
            mode='m# exportarkers',
            marker=dict(
                size=5,
                line_width=0.4,
                color=scatter_colors[i]
                )
            )
         )
                    
        gmm = generate_distributions(feat_vectors, len(centroid_ids))
        
        prototypes = feat_vectors[centroid_ids]
        
        fig.add_trace(go.Scatter(
            name=f'GMMs for {model}',
            x = prototypes[:, 0],
            y = prototypes[:, 1],
            mode='markers',
            marker=dict(
                size=10,
                line_width=0.8,
                color=gmm_colors[i]
                )
        ))
        
        gmms[model] = gmm
    
        fig = _visualize_gmm_2d(feat_vectors, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, fig,
                                 fixed_color=gmm_colors[i])
        
    fig.show()
    
    # Compute Kl divergence
    
    kl_div_d1_d2 = gmm_kl(gmms[model1], gmms[model2])
    kl_div_d2_d1 = gmm_kl(gmms[model2], gmms[model1])
    
    return kl_div_d1_d2, kl_div_d2_d1

In [ ]:
# export

def plot_kmedoids_decision_boundaries(cluster: Tuple, model: str,
                                       out_path: str,
                                       bound_h: Optional[float]=0.02,
                                       colorscale: Optional[str]="teal",
                                       mkrs_colors: Optional[List[str]]=["khaki", "lightcoral", "red"],
                                       img_fmt: Optional[str]="png",
                                       show: Optional[bool]=True,
                                       export: Optional[bool]=False):
    """
    Plot the decision boundaries for the clusters obtained via k-medoids
    
    :param cluster: Tuple containing cluster data
    :param model: Str identifying the model
    :param out_path: Str indicating the location for the exported plot
    :param bound_h: (Optional) float indicating the interpolation param for plotting the decision boundaries
    :param colorscale: (Optional) Str indicating the color for the decision boundaries
    :param mkrs_colors: (Optional) List indicating the colors for markers: data, centroids, criticisms
    :param img_fmt: (Optional) Str indicating the file format for the stored image
    :param show: (Optional) Bool indicating whether or not to show the plot
    :param export: (Optional) Bool indicating whether or not to export the plot
    """
    feat_vectors, centroids, xx, yy, Z, y_ = __setup_meshgrid(cluster, h=bound_h)
    _, crit_ids = gen_criticisms(feat_vectors, centroids)
    
    fig = go.Figure()
    
    boundaries_trace = go.Heatmap(x=xx[0], y=y_, z=Z,
                  colorscale=colorscale,
                  showscale=False)
    
    # Data points
    data_trace = go.Scatter(
        x=feat_vectors[:, 0],
        y=feat_vectors[:, 1],
        mode='markers',
        marker=dict(
            size=6,
            color=mkrs_colors[0],
            line=dict(color='black', width=0.2)
        ),
        name=f"{model} Data distribution"
    )
    
    # Prototypes (cluster centroids)
    proto_trace = go.Scatter(
        x=centroids[:, 0],
        y=centroids[:, 1],
        mode='markers',
        marker=dict(
            size=15,
            color=mkrs_colors[1],
            line=dict(color='black', width=1.2),
            symbol='octagon-dot'
        ),
        name="Prototypes"
    )        
    
    # Criticisms
    crit_points = np.take(feat_vectors, crit_ids, axis=0)
    
    crit_trace = go.Scatter(
        x = crit_points[:, 0],
        y = crit_points[:, 1],
        mode='markers',
        marker=dict(
            size=14,
            color=mkrs_colors[2],
            line=dict(color='black', width=0.9),
            symbol='x-dot'
        ),
        name="Criticisms"
    )
    
    fig.add_trace(boundaries_trace)
    fig.add_trace(data_trace)
    fig.add_trace(proto_trace)
    fig.add_trace(crit_trace)
    
    fig.update_layout(
        legend=dict(
            yanchor="top",
            xanchor="left",
        ),
        title=f'Prototypes & criticisims for {model}',
        xaxis=dict(
            title="UMAP Dim.1"
        ),
        yaxis=dict(
            title="UMAP Dim.2"
        )
    )
    
    if show:
        py.iplot(fig)
    
    if export:
        save_plotly_fig(fig, out_path, f"DecisionBoundaries-distr-{model}", img_fmt=img_fmt)                

In [64]:
# export


# Note the use of plot instead of scatter for clearer plots

def plot_data_distribution(feat_vectors: np.ndarray, model: str,
                             out_path: str,
                             color: Optional[str]='c.',
                             export: Optional[bool]=False):
    """
    Plot the data distribution of the provided vectors
    :param feat_vectors:
    :param model:
    :param out_path:
    :param color:
    :param export:
    
    """
    
    plt.figure(figsize=(8, 8))
    plt.title(f"Data Distribution for {model}")
    
    plt.plot(feat_vectors[:, 0], feat_vectors[:, 1], color, markersize = 4, zorder = -1)
    
    if export:
        _save_plot(out_path, f"Data-distribution-{model}")    

In [40]:
# export

def plot_gmm_2d(clusters: Dict, out_path: str, export = False):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(1, 2)
    plt.clf()
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))

        # Plot Data
        plt.subplot(gs[0, i])
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, i)
        
    if export:
        _save_plot(out_path, f"2D_GMM_{model}")
    plt.show()

In [38]:
# export

def plot_overlapping_features(clusters: Dict, 
                              out_path: str, colmap: Optional[Dict]={0: 'c.', 1: 'm.', 2: 'y.'},
                              lim: Optional[int]=10, export: Optional[bool]=False):
    """
    Plot the distribution for 2 sets of samples
    
    :param clusters: Dict containing cluster info. for 2 distributions
    :param out_path: Str indicating the location for storing generated figures
    :param colmap: Dict specifying colors for plots
    :param lim: int
    :param export: Bool indicating whether or not to store generated figure
    """
    plt.figure(figsize=(8, 8))
    plt.title(f'Overlapping features for {" and ".join(clusters.keys())} samples. \n')
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize = 4)
        
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
        
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.legend(clusters.keys())
    plt.grid(True)

    if export:
        _save_plot(out_path, f'overlapping_features_{"_".join(clusters.keys())}')

In [12]:
# export

# From http://www.itzikbs.com/gaussian-mixture-model-gmm-3d-point-cloud-classification-primer
def visualize_2D_gmm(points, w, mu, stdev, model_id, c, export=False):
    '''
    plots points and their corresponding gmm model in 2D
    Input: 
        points: N X 2, sampled points
        w: n_gaussians, gmm weights
        mu: 2 X n_gaussians, gmm means
        stdev: 2 X n_gaussians, gmm standard deviation (assuming diagonal covariance matrix)
    Output:
        None
    '''
    n_gaussians = mu.shape[1]
    N = int(np.round(points.shape[0] / n_gaussians))
    # Visualize data
    axes = plt.gca()
    colors = cmx.Accent(np.linspace(0, 1, n_gaussians)) if id == 'human' else cmx.Paired(np.linspace(0, 1, n_gaussians))
    for i in range(n_gaussians):
        idx = range(i * N, (i + 1) * N)
        for j in range(4):
            axes.add_patch(
                patches.Ellipse(mu[:, i], width=(j+1) * stdev[0, i], height=(j+1) *  stdev[1, i], alpha = 0.4, fill=True, color=colors[i]))
    plt.xlabel('Dim 1')
    plt.ylabel('Dim 2')

In [34]:
# export

def plot_gmm(cluster: Dict, model: str, c, out_path: str, export: Optional[bool]=True):
    """
    Plot Gaussian Mixture models for an individual set of samples
    
    :param cluster:
    :param model:
    :param c:
    :param out_path:
    :param export: Bool indicating whether or not store the generated figure
    """
    plt.figure(figsize = (8, 8))
    plt.title(f'Gaussian Mixture Models for {model} samples.\n')
    
    samples, _, medoid_ids, _ = cluster
    plt.plot(samples[:, 0], samples[:, 1], c, markersize = 4, zorder = -1)
    gmm = generate_distributions(samples, len(medoid_ids))

    # Plot Data
    visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, c)
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.legend([model])
    plt.grid(True)
    
    if export:
        _save_plot(out_path, f"gmm_{model}")

In [62]:
# export

def plot_kmedoids(cluster, model,
                  cmap = ListedColormap(["whitesmoke", "lightgrey", 'darkgrey', 'dimgrey', 'black']),
                  cm = 'c.', lim = 10, export = True):
    plt.figure(figsize=(8, 8))
    samples, centroids, xx, yy, Z = setup_meshgrid(cluster, lim)
    _, crit_ids = gen_criticisms(samples, centroids)
    plt.imshow(
        Z, interpolation='nearest',
        extent = (xx.min(), xx.max(), yy.min(), yy.max()),
        cmap = cmap, aspect = 'auto', origin = 'lower'
    )
    plt.plot(samples[:, 0], samples[:, 1], cm, markersize = 4)
    plt.scatter(
        centroids[:, 0], centroids[:, 1],
        marker = 'o', s = 64, linewidths = 3,
        color = 'b', zorder = 10
    )
    for crit_id in crit_ids:
        plt.scatter(
            samples[crit_id, 0], samples[crit_id, 1],
            marker = 'x', s = 64, linewidths = 3, color = 'r',
            zorder = 10
        )
    plt.title(
        f'K-Medoids clustering via\n'
        f'(PCA & T-SNE - reduced data) for {model} testbed\n'
    )
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.xlabel('T-SNE Dim 1')
    plt.ylabel('T-SNE Dim 2')
    plt.legend([model, 'prototypes', 'criticisms'])
    plt.grid(True)
    
    if export:
        plt.savefig(f'/tf/data/graphics/k-medoids_{model}.png', dpi=100, format='png')

Plotting related to GMM and Overlapping of distributions

In [42]:
# export

def plot_overlapping_gmm(clusters: Dict, out_path: str,
                         colmap: Optional = {0: 'c.', 1: 'm.', 2: 'y.'},
                         lim: Optional[float]=10.0, export: Optional[bool]=True):
    """
    Plot the comparison between 2 distributions: Includes overlapping distributions
    and GMMs
    
    :param clusters: Dict containing clusters data for 2 distributions
    :param colmap: Dict indicating colors for datapoints
    :param lim: Float indicating the limit value for the plots
    :param export: Bool indicating whether or not save the generated figure
    
    """
    plt.figure(figsize = (8, 8))
    plt.title(f'Gaussian Mixture Models for {" and ".join(clusters.keys())} samples.\n')

    gmms = {}
    cs = ['c', 'g', 'm', 'y']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize = 4, zorder = -1)
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.legend(clusters.keys())
    plt.grid(True)
    
    if export:
        _save_plot(out_path, f"overlapping_gmm_{'_'.join(clusters.keys())}")

In [63]:
def plot_gmm_single_distribution(cluster_data: Tuple, model: str,
                                   out_path: str, 
                                   colmap: Optional = {0: 'c.', 1: 'm.', 2: 'y.'},
                                   lim: Optional[float]=10.0,
                                   export: Optional[bool]=False):
    """
    Plot the distribution of data along with GMMs based on the provided clsuters
    
    :param cluster_data: Tuple containing (feat_vectors, _, cluster_centroids, _)
    :param model: Str indicating the model being plotted
    :param colmap: Dict. containing the colors
    :param lim: Float indicating the limit value for the plot
    :param export: Bool indicating whether or not save the generated plot
    """
    
    plt.figure(figsize = (8, 8))
    plt.title(f'Gaussian Mixture Models for {model}')
    gmms = {}
    feat_vectors, _, centroid_ids, _ = cluster_data
    plt.plot(feat_vectors[:, 0], feat_vectors[:, 1], 'c.', markersize = 4, zorder = -1)
    gmm = generate_distributions(feat_vectors, len(medoid_ids))
    gmms[model] = gmm
    
    visualize_2D_gmm(feat_vectors, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, 'g')
    
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    
    plt.grid(True)
    
    if export:
        _save_plot(out_path, f"GMM_distribution-{model}")

In [61]:
# export


def plot_overlapping_and_gmms(clusters: Dict, out_path: str, export: Optional[bool]=True) -> Tuple[float, float]:
    """
    Separately plot the overlapping distribution the GMM models for 2 given sample sets
    
    :param clusters: Dict. containing clusters info.
    :param export: Bool indicating whether or not store the generated figure
    
    :return: Tuple (KL div. dist1-dist2, KL div. dist2-dist1)    
    """
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
        
    plt.subplot(gs[1, 0])
    colmap = {0: 'co', 1: 'mo'}
    
    models = list(clusters.keys())
    plt.title(f'Overlapping distributions\nCyan denotes {models[0]} samples and \
    \nMagenta denotes {models[1]} samples.')
    
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize=2)
        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
    
    plt.subplot(gs[1, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    gmms = {}
    cs = ['c', 'm']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
        
    # Compute Kl divergence
    
    dist1_to_dist2_kl = gmm_kl(gmms[models[0]], gmms[models[1]])
    dist2_to_dist1_kl = gmm_kl(gmms[models[1]], gmms[models[0]])
    
    plt.title(f'{models[0]} to {models[1]} KL Divergence: {dist1_to_dist2_kl}\
    \n{models[1]} to {models[0]} KL Divergence: {dist2_to_dist1_kl}')
    
    plt.show()
    
    if export:
        _save_plot(out_path, f"Distributions-{models[0]}-vs-{models[1]}")
        
    return dist1_to_dist2_kl, dist2_to_dist1_kl

In [ ]:
# export


def plot_gmm_1d(clusters, export = False):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(1, 2)
    plt.clf()
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        x_min, x_max = samples[:, 0].min(), samples[:, 0].max()

        # Plot Data
        plt.subplot(gs[0, i])

        delta = 10
        x = np.linspace(x_min - delta, x_max + delta, 1000).reshape(500,2)
        logprob = gmm.score_samples(x)
        pdf = np.exp(logprob)
        plt.plot(x, pdf, '-k')
        
    if export:
        if not os.path.exists('images/'): os.mkdir('images/')
        plt.savefig('images/1D_GMM_demonstration.png', dpi=100, format='png')
        
    plt.show()

Plotting related to clustering

In [15]:
# export

def plot_features(clusters, export = True):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
    
    plt.subplot(gs[1, 0])
    colmap = {0: 'co', 1: 'mo'}
    plt.title('Overlapping Models\nCyan denotes Human Methods and\nMagenta denotes Generator Samples')
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize=2)
        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
    
    plt.subplot(gs[1, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    gmms = {}
    cs = ['c', 'm']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
    human_to_model_kl = gmm_kl(gmms['human'], gmms['model'])
    model_to_human_kl = gmm_kl(gmms['model'], gmms['human'])
    plt.title(f'Human to Model KL Divergence: {human_to_model_kl}\nModel to Human KL Divergence: {model_to_human_kl}')
    
    plt.show()
    
    if export:
        if not os.path.exists('images/'): os.mkdir('images/')
        plt.savefig('images/1D_GMM_demonstration.png', dpi=100, format='png')

In [51]:
# export

def plot_clusters(clusters: Iterable, out_path: str, export: Optional[bool]=True,
                  clustering_type: Optional[str]='K-medoids'):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
    for i, cluster in enumerate(clusters):
        # Setup data to be plotted
        feature_vectors, centroids, xx, yy, Z = setup_data(cluster)
        
        logger.info('Generated setup data (decision boundaries).')
        
        # Plot data
        plt.subplot(gs[0, i])
        plt.imshow(Z, interpolation='nearest',
               extent=(xx.min(), xx.max(), yy.min(), yy.max()),
               cmap=plt.cm.Paired,
               aspect='auto', origin='lower')

        plt.plot(feature_vectors[:, 0], feature_vectors[:, 1], 'k.', markersize=2)
        # Plot the centroids as a white X
        plt.scatter(centroids[:, 0], centroids[:, 1],
                    marker='x', s=169, linewidths=3,
                    color='w', zorder=10)
        plt.title(f'{clustering_type} Clustering\n'
                  '(PCA & T-SNE - reduced data)\n'
                  'Centroids are marked with white cross')
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        
    plt.subplot(gs[1, :])
    colmap = {0: 'b.', 1: 'r.'}
    plt.title('Blue denotes Human Methods and Red denotes Generator Samples')
    for i, cluster in enumerate(clusters):
        feature_vectors, _, _, _, _ = cluster
        plt.plot(feature_vectors[:, 0], feature_vectors[:, 1], colmap[i], markersize=10)
        
    if export:
        _save_plot(out_path, f"clustered_datapoints_")
        plt.savefig('images/feature_vectors_scatter_plot.png', dpi=100, format='png')
    plt.show()

In [58]:
from nbdev.export import notebook2script
notebook2script()

Converted 0.0_mgmnt.prep.i.ipynb.
Converted 0.1_mgmnt.prep.ipynb.
Converted 0.3_mgmnt.prep.bpe.ipynb.
Converted 0.6_mgmnt.prep.nltk.ipynb.
Converted 0.7_mgmnt.prep.files_mgmnt.ipynb.
Converted 0.8_mgmnt.prep.bpe_tokenization.ipynb.
Converted 1.0_exp.i.ipynb.
Converted 1.1_exp.info-[inspect].ipynb.
Converted 1.1_exp.info.ipynb.
Converted 1.2_exp.csnc.ipynb.
Converted 1.2_exp.gen.code.ipynb.
Converted 1.3_exp.csnc_python.ipynb.
Converted 10.0_utils.clusterization.ipynb.
Converted 10.1_utils.visualization.ipynb.
Converted 2.0_repr.codebert.ipynb.
Converted 2.0_repr.i.ipynb.
Converted 2.1_repr.codeberta.ipynb.
Converted 2.1_repr.roberta.train.ipynb.
Converted 2.2_repr.roberta.eval.ipynb.
Converted 2.3_repr.word2vec.train.ipynb.
Converted 2.6_repr.word2vec.eval.ipynb.
Converted 2.7_repr.distmetrics.ipynb.
Converted 2.8_repr.sentence_transformers.ipynb.
Converted 3.1_mining.unsupervised.traceability.eda.ipynb.
Converted 3.2_mining.unsupervised.eda.traceability.d2v.ipynb.
Converted 3.2_mutual